In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [10]:
# Load the dataset
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset["test"]

# Calculate the number of validation samples
num_validation_samples = 0.1 * mnist_info.splits["train"].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits["test"].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# Define the scaling function
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

# Apply the scaling function
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

# Shuffle the training and validation data
BUFFER_SIZE = 10000
shuffled_train_and_validation = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# Split the data into training and validation sets
validation_data = shuffled_train_and_validation.take(num_validation_samples)
train_data = shuffled_train_and_validation.skip(num_validation_samples)

# Batch the data
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(BATCH_SIZE)
test_data = test_data.batch(BATCH_SIZE)

# Define the model
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.Input(shape=(28, 28, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
    tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
    tf.keras.layers.Dense(output_size, activation="softmax")
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
NUM_EPOCHS = 5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=validation_data, verbose=2)

Epoch 1/5
540/540 - 11s - 21ms/step - accuracy: 0.8838 - loss: 0.4097 - val_accuracy: 0.9420 - val_loss: 0.2105
Epoch 2/5
540/540 - 3s - 5ms/step - accuracy: 0.9461 - loss: 0.1835 - val_accuracy: 0.9538 - val_loss: 0.1602
Epoch 3/5
540/540 - 3s - 5ms/step - accuracy: 0.9579 - loss: 0.1415 - val_accuracy: 0.9650 - val_loss: 0.1204
Epoch 4/5
540/540 - 3s - 5ms/step - accuracy: 0.9648 - loss: 0.1156 - val_accuracy: 0.9685 - val_loss: 0.1134
Epoch 5/5
540/540 - 3s - 5ms/step - accuracy: 0.9704 - loss: 0.0983 - val_accuracy: 0.9723 - val_loss: 0.0933


Testing:

In [11]:
test_loss, test_accuracy = model.evaluate(test_data)

100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9677 - loss: 0.1076


In [12]:
print("Test loss: {0: .2f}. Test accuracy: {1: .2f}%".format(test_loss, test_accuracy * 100.))

Test loss:  0.11. Test accuracy:  96.65%
